In [35]:
%load_ext autoreload
%autoreload 2
import sys

# instead of creating a package using setup.py or building from a docker/singularity file,
# import the sister directory of src code to be called on in notebook.
# This keeps the notebook free from code to only hold visualizations and is easier to test
# It also helps keep the state of variables clean such that cells aren't run out of order with a mysterious state
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import pandas as pd
import numpy as np
import gc

from src.processing import get_closest_radiometer, get_closest_nysm_stations
from src.data import nysm_data

In [37]:
nysm_df = nysm_data.load_nysm_data(gfs=True)
nysm_df.reset_index(inplace=True)
gc.collect()
nysm_df = nysm_df.rename(columns={"time_1H": "valid_time"})

/home/aevans/nwp_bias/src/machine_learning/notebooks/../src/data/nysm_data.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  nysm_1H_obs['snow_depth'].fillna(-999, inplace=True)
/home/aevans/nwp_bias/src/machine_learning/notebooks/../src/data/nysm_data.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

In [38]:
inc = []
for s in nysm_df["station"].unique():
    stations = get_closest_nysm_stations.get_closest_stations(nysm_df, 15, s, "GFS")
    inc.append(stations)

In [39]:
df = pd.DataFrame({"station": nysm_df["station"].unique(), "targets": inc})

In [40]:
df

,station,targets
0,ADDI,"[ADDI, ELMI, HART, TYRO, COHO]"
1,ANDE,"[ANDE, WALT, ROXB, CLAR, HARP]"
2,BATA,"[BATA, RUSH, BUFF, EAUR, BURT]"
3,BEAC,"[BEAC, BREW, HFAL, DOVE, SUFF]"
4,BELD,"[BELD, WALT, CINC, BERK, SHER]"
...,...,...
121,WOLC,"[WOLC, OSWE, JORD, CLIF, WATE]"
122,YORK,"[YORK, WARS, BROC, SBRI, GROV]"
123,CROG,"[CROG, OLDF, COPE, GFLD, EDWA]"
124,RAQU,"[RAQU, ILAK, NEWC, PISE, TUPP]"


In [41]:
df.to_csv(
    "/home/aevans/nwp_bias/src/machine_learning/data/gfs_data/gfs_stations_grouped.csv"
)

In [ ]:
station = "VOOR"

In [ ]:
stations_df = pd.read_parquet(
    "/home/aevans/nwp_bias/src/machine_learning/data/gfs_data/gfs_stations_grouped.parquet"
)
radiometer_df = pd.read_parquet(
    "/home/aevans/nwp_bias/src/machine_learning/data/profiler_images/profiler_stations_grouped.parquet"
)

stations_df = stations_df[stations_df["station"] == station]
radiometer_df = radiometer_df[radiometer_df["station"] == station]

stations = stations_df.iloc[0, 1]
stations